In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2022, 1, 14)

In [2]:
format_dict = {
               'qty':'{:,}','SHARES':'{:,}','available_qty':'{:,}','volbuy':'{:,}','volsell':'{:,}','volbal':'{:,}',
               'price':'{:,.2f}','buy_price':'{:,.2f}','PRICE':'{:,.2f}','max_price':'{:,.2f}','min_price':'{:,.2f}',
               'buy_target':'{:,.2f}','sell_target':'{:,.2f}','fee':'{:,.2f}','vat':'{:,.2f}',
               'net':'{:,.2f}','profit':'{:,.2f}','cost':'{:,.2f}','volume':'{:,.2f}','beta':'{:,.2f}',
               'Q4':'{:,.3f}','Q3':'{:,.3f}','Q2':'{:,.3f}','Q1':'{:,.3f}','DIVIDEND':'{:,.3f}',
               'percent':'{:,.2f}%','PERCENT':'{:,.2f}%','yearly':'{:,.2f}%','pct':'{:,.2f}%',
              }

### Tables in the process

In [3]:
# Sells table in MySQL portfolio database
sql = """
SELECT name, stock_id, B.date AS buy_date, qty, B.price AS buy_price, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

,name,stock_id,buy_date,qty,buy_price,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,SYNEX,113,2022-01-07,"1,500",31.50,904,1098,2022-01-13,33.00,102.47,7.17,"49,390.36",6,"2,035.70",4.30%,261.51%,96,None,0


### In case of sale, retreive buy id from sells record (buy_id)

In [4]:
# Buys table in MySQL portfolio database
transaction = 'S'

names = sells['name']
name = names.to_string(index=False)

sr_qty = sells['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1

sr_price = sells['buy_price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

sr_price = sells['price']
sell_price = sr_price.to_string(index=False)
sell_price = float(sell_price)

buy_ids = sells['buy_id']
buy_id = buy_ids.to_string(index=False)

sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = %s
"""
sql = sql % buy_id
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = 1098



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,SYNEX,1098,113,2022-01-07,"1,500",31.50,97.81,6.85,Sold,"47,354.66",DTD,None


In [5]:
transaction, name, qty, sell_price, buy_price

('S', 'SYNEX', -1500, 33.0, 31.5)

### In case of buy, retreive buy id from latest buy record (id)

In [6]:
sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1
"""
print(sql)
buys = pd.read_sql(sql, conpf)
buys


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,LPF,1099,176,2022-01-14,10000,14.0,289.8,20.29,Active,140310.09,DOS,None


In [7]:
transaction = 'B'

names = buys['name']
name = names.to_string(index=False)

sr_qty = buys['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty)

sr_price = buys['price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

transaction, name, qty, buy_price

('B', 'LPF', 10000, 14.0)

## Begin of Sale transaction

### Begin of Update buy table in MySQL stock database from sale transaction

In [ ]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy

In [ ]:
sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)

In [ ]:
rp = const.execute(sqlUpd)
rp.rowcount

In [ ]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy

### End of Update buy table in MySQL stock database from sale transaction

### Begin of Update dividend table in MySQL stock database from sale transaction

In [ ]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.style.format(format_dict)

In [ ]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)

In [ ]:
rp = const.execute(sqlUpd)
rp.rowcount

In [ ]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.style.format(format_dict)

### If shares become zero and no longer wants to keep dividend record

In [ ]:
sqlDel = """
DELETE FROM dividend
WHERE name = '%s' AND shares = 0
"""
sqlDel = sqlDel % name
print(sqlDel)

In [ ]:
rp = const.execute(sqlDel)
rp.rowcount

### End of Update dividend table in MySQL stock database from sale transaction

### Begin of Update stocks table in SQLite port_lite database from sale transaction

In [ ]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)

In [ ]:
sell_target = 37.00 # must calculate manually beforehand
#reason = "15pct"

sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s,
sell_target = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, sell_target, name)
print(sqlUpd)

In [ ]:
rp = conlite.execute(sqlUpd)
rp.rowcount

In [ ]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)

### End of Update stocks table in SQLite port_lite database from sale transaction

## End of Sale process

## Begin of  Buy process

### Begin of Update buy table in MySQL database from buy transaction

In [5]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy


SELECT * 
FROM buy 
WHERE name = 'LPF'



,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade


### In case there is the same stock name in port

In [ ]:
new_cost = 40.50 # new average cost must be calculated manually beforehand

sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)

In [ ]:
rp = const.execute(sqlUpd)
rp.rowcount

In [ ]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy

### In case of totally new stock

In [6]:
sqlIns = """
INSERT INTO buy 
VALUES('LPF','2022-01-14',10000,14,0,0,1,0,'4','A3')
"""
print(sqlIns)


INSERT INTO buy 
VALUES('LPF','2022-01-14',10000,14,0,0,1,0,'4','A3')



In [7]:
#rp = const.execute(sqlIns)
rp.rowcount

1

### End of Update buy table in MySQL database from buy transaction

### Begin of Update dividend table in MySQL stock database from buy transaction

In [8]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'LPF' 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,LPF,0.200,0.189,0.179,0.188,0.756,0.00,0.00%,0.0,2021-11-12,2021-11-12,,0


In [9]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + 10000
WHERE name = 'LPF'



In [10]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [11]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'LPF' 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,LPF,0.200,0.189,0.179,0.188,0.756,0.00,0.00%,"10,000.0",2021-11-12,2021-11-12,,0


### End of Update dividend table in MySQL stock database from buy transaction

### Begin of Update stocks table in SQLite port_lite database from buy transaction

In [16]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)


SELECT * 
FROM stocks 
WHERE name = 'LPF'



In [18]:
stocks = pd.read_sql(sql, conlite)
stocks

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,690,LPF,0,0,I,0,0,None,None,0,10000,-4,4,0,0,0,DOS,SET


In [19]:
new_cost = 14.00
sell_target = 14.30
status = 'I'
reason = 'DOS'
new_sale_qty = 10000
new_cost, sell_target, status, reason, new_sale_qty

(14.0, 14.3, 'I', 'DOS', 10000)

In [20]:
sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s, 
cost = %s, sell_target = %s, status = '%s', 
reason = '%s', qty = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, sell_target, status, reason, new_sale_qty, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + 10000, 
cost = 14.0, sell_target = 14.3, status = 'I', 
reason = 'DOS', qty = 10000
WHERE name = 'LPF'



In [21]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [24]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'LPF'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,690,LPF,0.00,0.00,I,0.00,14.30,0.00,0.00,14.00,"10,000",-4,4,"10,000",0,0,DOS,SET


### End of Update stocks table in SQLite port_lite database from buy transaction

## End of  Buy process